In [ ]:
import numpy as np
import tkinter as tk
from tkinter import messagebox, scrolledtext
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

class MatrixSolverApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Matrix Solver")
        
        # Frame for input and buttons
        self.frame_input = tk.Frame(self.root,bg='lightblue')
        self.frame_input.pack(padx=20, pady=20)
        
        # Label and entry for number of rows
        self.label_rows = tk.Label(self.frame_input, text="Number of Rows:")
        self.label_rows.grid(row=0, column=0, padx=10, pady=5)
        self.entry_rows = tk.Entry(self.frame_input)
        self.entry_rows.grid(row=0, column=1, padx=10, pady=5)
        
        # Label and entry for number of columns
        self.label_cols = tk.Label(self.frame_input, text="Number of Columns:")
        self.label_cols.grid(row=1, column=0, padx=10, pady=5)
        self.entry_cols = tk.Entry(self.frame_input)
        self.entry_cols.grid(row=1, column=1, padx=10, pady=5)
        
        # Button to input matrix
        self.button_input_matrix = tk.Button(self.frame_input, text="Input Matrix", command=self.input_matrix)
        self.button_input_matrix.grid(row=2, columnspan=2, padx=10, pady=10)
        
        # Frame for displaying results
        self.frame_results = tk.Frame(self.root)
        self.frame_results.pack(padx=20, pady=2)
        
        # Canvas for matplotlib plots
        self.fig = Figure(figsize=(6, 4))
        self.ax = self.fig.add_subplot(111, projection='3d')
        self.ax.set_title("Solution Plot")
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.frame_results)
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)
        
        # Text display for solutions
        self.text_display = scrolledtext.ScrolledText(self.frame_results, width=40, height=10)
        self.text_display.pack(pady=10)
        
        # Button to solve and plot
        self.button_solve_plot = tk.Button(self.frame_results, text="Solve and Plot", command=self.solve_and_plot,bg='blue')
        self.button_solve_plot.pack(pady=10)
        
        # Initialize matrix variables
        self.matrix = None
        self.A = None
        self.b = None
    
    def input_matrix(self):
        try:
            rows = int(self.entry_rows.get())
            cols = int(self.entry_cols.get())
            if rows <= 0 or cols <= 0:
                raise ValueError("Number of rows and columns must be positive integers.")
            
            self.matrix = np.zeros((rows, cols))
            
            # Create input matrix dialog
            self.input_dialog = tk.Toplevel()
            self.input_dialog.title("Input Matrix")
            
            # Create input entries for each matrix element
            self.entries = []
            for i in range(rows):
                row_entries = []
                for j in range(cols):
                    label = tk.Label(self.input_dialog, text=f"Element ({i+1},{j+1}):")
                    label.grid(row=i, column=2*j, padx=5, pady=5)
                    entry = tk.Entry(self.input_dialog, width=10)
                    entry.grid(row=i, column=2*j+1, padx=5, pady=5)
                    row_entries.append(entry)
                self.entries.append(row_entries)
            
            # Button to confirm input
            button_confirm = tk.Button(self.input_dialog, text="Confirm", command=self.confirm_input)
            button_confirm.grid(row=rows, columnspan=cols, padx=10, pady=10)
        
        except ValueError as e:
            messagebox.showerror("Error", str(e))
    
    def confirm_input(self):
        try:
            rows, cols = self.matrix.shape
            for i in range(rows):
                for j in range(cols):
                    self.matrix[i, j] = float(self.entries[i][j].get())
            
            self.input_dialog.destroy()
            self.A = self.matrix[:, :-1]
            self.b = self.matrix[:, -1]
            
            messagebox.showinfo("Success", "Matrix input successful.")
        
        except ValueError:
            messagebox.showerror("Error", "Invalid matrix element(s). Please enter numeric values.")
    
    def solve_homogeneous(self):
        if self.A is not None:
            U, s, Vh = np.linalg.svd(self.A)
            null_space = Vh[s.size:].T
            return null_space
        return None
    
    def solve_nonhomogeneous(self):
        if self.A is not None and self.b is not None:
            try:
                solution = np.linalg.solve(self.A, self.b)
                return solution
            except np.linalg.LinAlgError:
                return "No unique solution exists"
        return None
    
    def plot_solution_homogeneous(self, solution):
        if solution is not None and solution.shape[1] == 3:
            self.ax.clear()
            x = np.linspace(-10, 10, 400)
            y = np.linspace(-10, 10, 400)
            X, Y = np.meshgrid(x, y)
            Z = (-solution[0] * X - solution[1] * Y) / solution[2]
            self.ax.plot_surface(X, Y, Z, alpha=0.5)
            self.ax.set_title("Homogeneous Solution")
            self.canvas.draw()
    
    def plot_solution_nonhomogeneous(self, solution):
        if solution is not None and isinstance(solution, np.ndarray) and solution.shape[0] == 3:
            self.ax.clear()
            self.ax.scatter(solution[0], solution[1], solution[2], c='r', marker='o')
            self.ax.set_title("Nonhomogeneous Solution")
            self.canvas.draw()
    
    def solve_and_plot(self):
        homogeneous_solution = self.solve_homogeneous()
        nonhomogeneous_solution = self.solve_nonhomogeneous()
        
        # Clear previous text
        self.text_display.delete(1.0, tk.END)
        
        # Display solutions in text area
        self.text_display.insert(tk.END, "Homogeneous Solution:\n")
        if homogeneous_solution is not None:
            self.text_display.insert(tk.END, str(homogeneous_solution) + "\n")
        else:
            self.text_display.insert(tk.END, "No solution\n")
        
        self.text_display.insert(tk.END, "\nNonhomogeneous Solution:\n")
        if isinstance(nonhomogeneous_solution, np.ndarray):
            self.text_display.insert(tk.END, str(nonhomogeneous_solution) + "\n")
        else:
            self.text_display.insert(tk.END, str(nonhomogeneous_solution) + "\n")
        
        # Plot solutions
        self.plot_solution_homogeneous(homogeneous_solution)
        self.plot_solution_nonhomogeneous(nonhomogeneous_solution)

# Create tkinter root window and run the app
root = tk.Tk()
app = MatrixSolverApp(root)
root.mainloop()
